<a href="https://colab.research.google.com/github/brokoli777/Emotion-Recogntion/blob/main/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Import libraries for data manipulation, visualization and warnings
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random

# Import tensorflow and keras for deep learning tools
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential
from keras.layers import (
    Input,
    Dense,
    Conv2D,
    Dropout,
    Flatten,
    MaxPooling2D,
    BatchNormalization,
    Activation,
    LeakyReLU,
    GlobalAveragePooling2D,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

NUM_CLASSES = 7
TARGET_SIZE = (48, 48)  # Target image size for resizing
INPUT_SHAPE = (48, 48, 1)  # grayscale image

In [14]:
# Import kagglehub to download datasets from Kaggle
import kagglehub

# Download FER2013 dataset from Kaggle (didplay path for referrence)
path = kagglehub.dataset_download("msambare/fer2013")
print("Path to dataset files:", path)

100%|██████████| 60.3M/60.3M [00:02<00:00, 30.6MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/msambare/fer2013/versions/1


In [15]:
# Import shutil library for file operations
import shutil

# Define source and destination paths for dataset
source = "/root/.cache/kagglehub/datasets/msambare/fer2013/versions/1"
destination = "/content/dataset"

# Move dataset files from source to destination folder (display where moved to for referrence)
shutil.move(source, destination)
print(f"Files moved to {destination}")

Error: Destination path '/content/dataset/1' already exists

In [ ]:
# Define the paths for training and testing directories
TRAIN_DIR = './dataset/train'
TEST_DIR = './dataset/test'

In [ ]:
# Display the contents of the dataset directories
print(os.listdir(TRAIN_DIR))
print(os.listdir(TEST_DIR))
print(len(os.listdir(TRAIN_DIR+'/angry')))
print(len(os.listdir(TRAIN_DIR+'/disgust')))

In [ ]:
# Function to load the dataset images and their labels
def load_dataset(directory):
    image_paths = []  # List for storing image paths
    labels = []       # List for storing labels

    # Iterate through each label in the dataset directory
    for label in os.listdir(directory):
        label_path = os.path.join(directory, label)  # Use os.path.join to create label path
        if not os.path.isdir(label_path):  # Skip if it's not a directory
            continue

        # Iterate through the images within the label directory
        for filename in os.listdir(label_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                image_path = os.path.join(label_path, filename)
                image_paths.append(image_path)  # Append image path
                labels.append(label)            # Append label

        # Display completion validation
        print(label, "Completed")

    return image_paths, labels

In [ ]:
# Convert training dataset into a DataFrame
train = pd.DataFrame()
train['image'], train['label'] = load_dataset(TRAIN_DIR)

# Mix up the training dataset
# set seed for reproducibility
train = train.sample(frac=1, random_state=42).reset_index(drop=True)
train.head()


test = pd.DataFrame()
test['image'], test['label'] = load_dataset(TEST_DIR)
test = test.sample(frac=1, random_state=42).reset_index(drop=True)
test.head()

In [ ]:
for label in os.listdir(TRAIN_DIR):
    label_path = os.path.join(TRAIN_DIR, label)
    if os.path.isdir(label_path):
        print(f"{label}: {len(os.listdir(label_path))} images")

In [ ]:
def get_features(image_paths, target_size=(48, 48), color_mode='grayscale'):
    img_features = []
    for image_path in image_paths:
        if not os.path.exists(image_path):
            import sys
            print(f"Image not found: {image_path}", file=sys.stderr)
            continue
        try:
            img = load_img(image_path, target_size=target_size, color_mode=color_mode)
            img = img_to_array(img) / 255.0  # Normalize
            img_features.append(img)
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
    return np.array(img_features).reshape(-1, *target_size, 1)


In [ ]:
def validate_features(features, target_shape=(48, 48, 1)):

    if features.shape[1:] != target_shape:
        print(f"Validation failed: Expected shape {target_shape}, but got {features.shape[1:]}.")
        return False

    if np.any(np.isnan(features)) or np.any(np.isinf(features)):
        print("Validation failed: Found NaN or infinite values in features.")
        return False

    if np.min(features) < 0.0 or np.max(features) > 1.0:
        print("Validation failed: Feature values are not in the range [0, 1].")
        return False

    print("Validation passed: All checks passed.")
    return True

In [ ]:
# Extract the features from training and testing datasets
TARGET_SIZE = (48, 48)
train_features = get_features(train['image'],  target_size=TARGET_SIZE)
test_features = get_features(test['image'], target_size=TARGET_SIZE)

# validation of features train & test

if validate_features(train_features) and validate_features(test_features):
    print("Features validation passed.")
else:
    print("Features validation failed.")


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder to training labels
label_encoder.fit(train["label"])

# Transform both training and testing labels to integer values
# 0 to 6 for the 7 classes start from angry to neutral
y_train = label_encoder.transform(train["label"])
y_test = label_encoder.transform(test["label"])

# Convert integer labels to one-hot encoded vectors using Keras
# Dynamic exact number of classes
num_classes = len(label_encoder.classes_)

"""
To convert the integer labels to one-hot encoded vectors, we can use the to_categorical function from Keras.
This function takes the integer labels and the number of classes as input and returns the one-hot encoded vectors.
"""

y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

# Output the one-hot encoded vectors and classes
print("One-hot encoded labels (first 5 rows):")
print(y_train[:5])
print("Classes:")
print(label_encoder.classes_)

In [ ]:
x_train = train_features
x_test = test_features
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(np.argmax(y_train, axis=1)),
    y=np.argmax(y_train, axis=1)
)
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
# Build the CNN model
model = Sequential()
model.add(Input(shape=INPUT_SHAPE))

# First convolutional block
# - The 32 filters help the model learn 32 distinct patterns or features from the input data.
# - The kernel size (3x3) determines the receptive field used to scan the input.
# - ReLU (Rectified Linear Unit) activation introduces non-linearity to enable the model
#   to learn complex relationships between features.
# This layer extracts low-level features such as edges and textures from the input image.
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))

# Add a batch normalization layer immediately after the convolutional layer.
# - This layer normalizes the outputs of the convolutional layer to have a mean of 0
#   and a standard deviation of 1, which helps stabilize the learning process.
# - It reduces sensitivity to weight initialization and allows the model to train faster.
# Batch normalization also acts as a regularizer, reducing the chances of overfitting.
model.add(BatchNormalization())

# Second convolutional block
# - Increase filters to 64
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

# - Add a max pooling layer with a pool size of 2x2.
# - This layer downsamples the spatial dimensions of the feature maps,
# - reducing computation and helping the model focus on dominant features.
model.add(MaxPooling2D(pool_size=(2, 2)))

# - Add a dropout layer with a dropout rate of 0.5.
# - This randomly sets 50% of the neurons to zero during training to prevent overfitting
# - and improve the generalization capability of the model.
model.add(Dropout(0.5))

# Third convolutional block
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Fourth convolutional block
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# Fully connected layers
model.add(Flatten())  # Flatten feature maps

# - The large number of neurons (1024) provides the model with significant capacity
# - to learn intricate details in the data.
model.add(Dense(1024, activation='relu')) # Fully connected layer
model.add(Dropout(0.2)) # Dropout

# - Each neuron corresponds to one of the 7 emotion classes in the dataset.
# - The softmax activation function ensures that the output values represent probabilities
# - and sum to 1, making it suitable for multi-class classification tasks.
model.add(Dense(7, activation='softmax')) # Output layer

# Display the model summary
model.summary()

# Complie the model using Adam optimizer and categorical cross-entropy loss
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model2 = Sequential()
model2.add(Input(shape=INPUT_SHAPE))

# Convolutional Input Layer
model2.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu"))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.3))

# Convolutional HL 2
model2.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation="relu"))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.3))

# Convolutional HL 3
model2.add(Conv2D(128, kernel_size=(3, 3), padding="same", activation="relu"))
model2.add(BatchNormalization())
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Dropout(0.4))

# Global Average Pooling
model2.add(GlobalAveragePooling2D())
model2.add(Dense(64, activation="relu"))
model2.add(Dropout(0.3))

# Output Layer
model2.add(Dense(7, activation="softmax"))

# Compile the model
opti = Adam(learning_rate=0.0005)
model2.compile(optimizer=opti, loss="categorical_crossentropy", metrics=["accuracy"])
model2.summary()

- Validation Loss Fluctuations:
- Validation loss fluctuates significantly (e.g., jumping from 1.8796 to 4.0646 and then back to 1.8438), indicating potential overfitting or instability in training.

- Drop in Performance in Later Epochs:
	•	After an initial improvement, performance metrics degrade in some epochs (e.g., validation loss increases to 3.9991 in epoch 5), possibly due to overfitting, inadequate regularization, or a mismatch in data preprocessing between training and validation datasets.

In [ ]:
# Setup for early stopping and learning rate reduction callbacks
# - Callback stops training early if validation loss does not improve
# - within given number of epochs (50)
early_stopping = EarlyStopping("val_loss", patience=50)

# Define the ReduceLROnPlateau callback.
# - This reduces the learning rate when the validation loss stops improving.
# - `monitor='val_loss'`: Monitors the validation loss during training.
# - `factor=0.1`: Reduces the learning rate by a factor of 10 when triggered.
# - `patience=int(50/4)`: Waits for approximately 12 epochs (1/4 of `patience` for early stopping) without improvement before reducing the learning rate.
# - `verbose=1`: Logs messages to the console when the learning rate is reduced.
reduce_lr = ReduceLROnPlateau("val_loss", factor=0.1, patience=int(50 / 4), verbose=1)

# Train the model
# - `x=x_train, y=y_train`: The training data and corresponding labels.
# - `batch_size=64`: Specifies the number of training samples processed before the model updates.
# - `epochs=40`: The maximum number of times the entire training dataset is processed.
# - `verbose=1`: Displays detailed progress information during training.
# - `validation_data=(x_test, y_test)`: Specifies the validation dataset for monitoring validation performance.
# - `callbacks=[early_stopping, reduce_lr]`: Adds the early stopping and learning rate reduction callbacks
# - to dynamically manage training termination and learning rate adjustments.

if len(train_features) != len(y_train):
    print("Length of train_features and y_train are not equal")
else:
    model_info = model.fit(
        x=x_train,
        y=y_train,
        epochs=40, # 40 epochs finer tuning
        batch_size=128,
        verbose=1, # display progress
        validation_data=(x_test, y_test),
        callbacks=[early_stopping, reduce_lr],
    )

In [ ]:
model_info2 = model2.fit(
    x=x_train,
    y=y_train,
    epochs=40,
    verbose=1,
    batch_size=128,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping, reduce_lr],
)

In [ ]:
# Evaluate the accuracy
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"Model 1: Test loss: {loss:.4f}, accuracy: {accuracy:.4f}")

loss2, accuracy2 = model2.evaluate(x_test, y_test, verbose=0)
print(f"Model 2: Test loss: {loss2:.4f}, accuracy: {accuracy2:.4f}")


In [ ]:
# Save the model structure and weights
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# Save the trained model weight in .h5 file
model.save_weights('model.weights.h5')

In [ ]:
# Save the complete model in .keras format
model.save("finalmodel.keras")

In [ ]:
# Plot the training accuracy and loss graphs
accuracy = model_info.history['accuracy']
val_accuracy = model_info.history['val_accuracy']
loss = model_info.history['loss']
val_loss = model_info.history['val_loss']

accuracy2 = model_info2.history['accuracy']  # Training accuracy for Model 2
val_accuracy2 = model_info2.history['val_accuracy']  # Validation accuracy for Model 2
loss2 = model_info2.history['loss']  # Training loss for Model 2
val_loss2 = model_info2.history['val_loss']  # Validation loss for Model 2

In [ ]:

# Plot accuracy comparison
plt.figure(figsize=(14, 6))
epochs = range(1, len(accuracy) + 1)

plt.subplot(1, 2, 1)
plt.plot(epochs, accuracy, label='Model 1 - Accuracy', linestyle='--')
plt.plot(epochs, val_accuracy, label='Model 1 - Val Accuracy', linestyle='--')
plt.plot(epochs, accuracy2, label='Model 2 - Accuracy', linestyle='-')
plt.plot(epochs, val_accuracy2, label='Model 2 - Val Accuracy', linestyle='-')
plt.title('Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss comparison
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, label='Model 1 - Loss', linestyle='--')
plt.plot(epochs, val_loss, label='Model 1 - Val Loss', linestyle='--')
plt.plot(epochs, loss2, label='Model 2 - Loss', linestyle='-')
plt.plot(epochs, val_loss2, label='Model 2 - Val Loss', linestyle='-')
plt.title('Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Test the model on a random image from the test set
image_index = random.randint(0, len(test))  # Pick image using random index number
print("Original Output:", test['label'][image_index]) # Display original label

# Predict the label using trained model
pred = model.predict(x_test[image_index].reshape(1, 48, 48, 1)) # Reshape to mathc model dimensions

# Decode predicted label back to original string with LabelEncoder
prediction_label = label_encoder.inverse_transform([pred.argmax()])[0]
print("Predicted Output:", prediction_label) # Display predicted label

# Display test image in grayscale to see input
plt.imshow(x_test[image_index].reshape(48, 48), cmap='gray');